In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Импорт нужных модулей.

In [ ]:
from tensorflow.keras.layers import *
from tensorflow.keras.metrics import *
from tensorflow.keras.models import Sequential, Model
import numpy as np
import pandas as pd
import os
import tensorflow.keras.backend as K
from sklearn.metrics import *
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing
from sklearn.preprocessing import MinMaxScaler
import pickle

Объявление переменных

In [ ]:
path = '/content/drive/MyDrive/CICIDS2017/data_nn'

data = []
target = []

Чтение датасета.

In [ ]:
for name in os.listdir(path):
    file = pd.read_csv(os.path.join(path, name))
    for n, i in file.iterrows():
        a = []
        for j in i[:-1]:
          a.append(j)
        data.append(a)

        target.append(0 if i[-1] == 'BENIGN' else 1)
del file

Обработка данных

In [ ]:
data = np.array(data)
target = np.array(target)
print(data.shape, target.shape)

(2830743, 78) (2830743,)


Подготовка данных для нормализации

In [ ]:
scaler = MinMaxScaler(feature_range=(-1,1))
scaler.fit(np.nan_to_num(data).astype(float))
data = scaler.transform(np.nan_to_num(data).astype(float))
data = data.reshape(data.shape[0], 78, 1)

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:87: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:87: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


Сохранение нормализатора

In [ ]:
pickle.dump(scaler, open('/content/drive/MyDrive/scaler.pkl', 'wb'))

Моделирование нейросети

Difference betwen backup:
1. reccurent activation = 'relu' (sigmoid)
2. range (4) ((3))

In [ ]:
lstm = Sequential()
lstm.add(Input((78, 1)))
lstm.add(LSTM(32, recurrent_activation='relu', return_sequences=True, recurrent_dropout=0))
for i in range(4):
  lstm.add(LSTM(32, recurrent_activation='relu', return_sequences=True, recurrent_dropout=0))
lstm.add(Flatten())
lstm.add(Dense(2,activation = 'softmax'))
lstm.compile(optimizer='adam', loss='binary_crossentropy', metrics='accuracy')

Тренировка нейросети

In [ ]:
lstm.fit(data, to_categorical(target), epochs=10, batch_size=256, verbose=1)
lstm.save('/content/drive/MyDrive/lstm.h5')

Epoch 1/10
11058/11058 [==============================] - 3579s 324ms/step - loss: 0.4962 - accuracy: 0.8030
Epoch 2/10
11058/11058 [==============================] - 3568s 323ms/step - loss: 0.4962 - accuracy: 0.8030
Epoch 3/10
11058/11058 [==============================] - 3632s 328ms/step - loss: 0.4962 - accuracy: 0.8030
Epoch 4/10
11058/11058 [==============================] - 3634s 329ms/step - loss: 0.4962 - accuracy: 0.8030
Epoch 5/10
11058/11058 [==============================] - 3618s 327ms/step - loss: 0.4962 - accuracy: 0.8030
Epoch 6/10
11058/11058 [==============================] - 3605s 326ms/step - loss: 0.4962 - accuracy: 0.8030
Epoch 7/10
11058/11058 [==============================] - 3577s 323ms/step - loss: 0.4962 - accuracy: 0.8030
Epoch 8/10
11058/11058 [==============================] - 3539s 320ms/step - loss: 0.4962 - accuracy: 0.8030
Epoch 9/10
11058/11058 [==============================] - 3517s 318ms/step - loss: 0.4962 - accuracy: 0.8030
Epoch 10/10
 6108/1

Получение данных от нейросети

In [ ]:
y_pred1 = lstm.predict(data, batch_size=1024)
y_pred = np.argmax(y_pred1, axis=1)

Вычисление метрик и количества пакетов с атакой и без

In [ ]:
print('recall:', recall_score(target, y_pred, average='micro'))
print('precision:', precision_score(target, y_pred, average='micro'))
print('accuracy:', accuracy_score(target, y_pred))
print('f1 score:', f1_score(target, y_pred, average='micro'))
print('ROC AUC:', roc_auc_score(target, y_pred))
print('матрица неточностей', confusion_matrix(target, y_pred))
unique, counts = np.unique(y_pred, return_counts=True)
count_list = dict(zip(unique, counts))
print('Нейросеть решила, что', count_list[1], 'пакетов это атака')
print('А', count_list[0], 'не атака')

recall: 0.876659944050025
precision: 0.876659944050025
accuracy: 0.876659944050025
f1 score: 0.876659944050025
ROC AUC: 0.7092836086821249
матрица неточностей [[2240089   33008]
 [ 316136  241510]]
Нейросеть решила, что 274518 пакетов это атака
А 2556225 не атака


Чтение датасета для второй нейросети. KNN

In [ ]:
data = []
target = []
last = []
for name in os.listdir(path):
    file = pd.read_csv(os.path.join(path, name))
    for n, i in file.iterrows():
        if not i[-1] in last:
            last.append(i[-1])
        if last.index(i[-1]) != 0:
          a = []
          for j in i[:-1]:
              a.append(j)
          data.append(a)
          target.append(last.index(i[-1])-1)
del file
del last[0]

Обработка

In [ ]:
data = np.array(data)
target = np.array(target)
data = data.reshape(data.shape[0], 78)
print(data.shape, target.shape)

Загрузка нормализатора

In [ ]:
scaler = pickle.load(open('/content/drive/MyDrive/scaler.pkl', 'rb'))

Нормализация данных для второй нейросети

In [ ]:
data = scaler.transform(np.nan_to_num(data).astype(float))

Моделирование второй нейросети

In [ ]:
knn = Sequential()
knn.add(InputLayer((78,)))
knn.add(BatchNormalization())
knn.add(Dense(128, activation='relu'))
knn.add(Dense(128, activation='relu'))
knn.add(Dense(14, activation='softmax'))
#optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
knn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy')

Тренировка второй нейросети

In [ ]:
knn.fit(data.astype(float), target, epochs=10, batch_size=256, verbose=1)

knn.save('/content/drive/MyDrive/knn.h5')

Получение данных со второй нейросети

In [ ]:
y_pred1 = knn.predict(data)
y_pred = np.argmax(y_pred1, axis=1)

Вычисление метрик и количество классов

In [ ]:
print('recall:', recall_score(target, y_pred, average='micro'))
print('precision:', precision_score(target, y_pred, average='micro'))
print('accuracy:', accuracy_score(target, y_pred))
print('f1 score:', f1_score(target, y_pred, average='micro'))
print('ROC AUC:', roc_auc_score(to_categorical(target), np.nan_to_num(y_pred1), multi_class='ovr'))
print('матрица неточностей', confusion_matrix(target, y_pred))
unique, counts = np.unique(y_pred, return_counts=True)
count_list = dict(zip(unique, counts))
print('Классы:')
for i in count_list:
  print(last[i], count_list[i])

recall: 0.9976920842254764
precision: 0.9976920842254764
accuracy: 0.9976920842254764
f1 score: 0.9976920842254764
ROC AUC: 0.9997288439737718
матрица неточностей [[127926      2      7      0      0      0      0      0      1      0
       0     91      0      0]
 [     0 158836      0      0      9      0      0      0      4     18
       3     58      2      0]
 [     0      0   1959      0      0      0      0      0      1      0
       0      6      0      0]
 [     1      0      1     33      0      0      0      0      1      0
       0      0      0      0]
 [     0      3      0      0   1359      1      0      0    143      0
       0      0      1      0]
 [     0      2      0      0    607     16      0      0      6      0
       0     17      4      0]
 [     0      0      0      0      4      0      0      0      8      0
       0      1      8      0]
 [     0      1      0      0      1      0      0   7912      6      7
       0     11      0      0]
 [     0     

-

-

-

Здесь начинается проверка нейросети

In [ ]:
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd

Загрузка заранее обученной нейросети

In [ ]:
lstm = load_model('/content/drive/MyDrive/lstm.h5')
knn = load_model('/content/drive/MyDrive/knn.h5')

Загрузка данных для тестирования нейросети

In [ ]:
file = pd.read_csv('/content/drive/MyDrive/net/Tuesday-WorkingHours.pcap_ISCX.csv')
data = []
target = []

for n, i in file.iterrows():
        a = []
        for j in i[:-1]:
          a.append(j)
        data.append(a)
        target.append(last.index(i[-1]))

Обработка данных

In [ ]:
data = np.array(data)
target = np.array(target)
data = data.reshape(data.shape[0], 78, 1)
print(data.shape, target.shape)

Проверка нейросети

In [ ]:
model.evaluate(data, target)

Предсказание нейросети

In [ ]:
pred = model.predict(data)

Определение класса предсказанния нейросети

In [ ]:
print(last[np.argmax(pred[5])])

NameError: ignored